In [1]:
import re
import pandas as pd
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
metadata = pd.read_csv('meta.csv')
metadata.drop(['price', 'imUrl', 'salesRank', 'related'], axis = 1, inplace = True)

ratings = pd.read_csv('ratings_Musical_Instruments.csv', names = ['reviewerID', 'asin', 'rating', 'unixReviewTime'])
del ratings['unixReviewTime']

df = pd.merge(ratings, metadata, on = 'asin')
df.drop(['reviewerID', 'asin'], axis = 1, inplace = True)
print(df)

        rating                                              title  \
0          3.0  Six Sonatas For Two Flutes Or Violins, Volume ...   
1          5.0  Double Concerto in D Minor By Johann Sebastian...   
2          5.0  Hal Leonard Vivaldi Four Seasons for Piano (Or...   
3          5.0  Aida: Opera in Quattro Atti, Partitura -- Aida...   
4          1.0                                          Nocturnes   
5          5.0                 Sisyphus Redux for solo alto flute   
6          3.0  Best of Stevie Ray Vaughan Signature Licks Gui...   
7          2.0      Hal Leonard Eric Clapton - The Solo Years DVD   
8          1.0      Hal Leonard Eric Clapton - The Solo Years DVD   
9          5.0      Hal Leonard Eric Clapton - The Solo Years DVD   
10         2.0   Hal Leonard Eric Clapton - The Early Years (DVD)   
11         2.0   Hal Leonard Eric Clapton - The Early Years (DVD)   
12         5.0   Hal Leonard Eric Clapton - The Early Years (DVD)   
13         5.0   Hal Leonard Eric 

In [3]:
guitarOnly = df[df['title'].str.contains('guitar', na = False)]
guitarOnly = guitarOnly[df['categories'].apply(str).str.contains('Guitars' or 'Stringed')]
guitarOnly['titleDesc'] = guitarOnly['title'] + ' ' + guitarOnly['description']
guitarOnly.drop(['title', 'categories', 'brand', 'description'], axis = 1, inplace = True)
guitarOnly = guitarOnly.dropna().reset_index(drop = True)
print(guitarOnly)

    rating                                          titleDesc
0      5.0  Fender blue ashbory fretless elecric bass guit...
1      4.0  Fender blue ashbory fretless elecric bass guit...
2      2.0  Fender blue ashbory fretless elecric bass guit...
3      4.0  Seavey Electronics This junior acoustic guitar...
4      5.0  Seavey Electronics This junior acoustic guitar...
5      5.0  Seavey Electronics This junior acoustic guitar...
6      4.0  Seavey Electronics This junior acoustic guitar...
7      4.0  Darling Divas Acoustic Steel String Guitar Pac...
8      5.0  Darling Divas Acoustic Steel String Guitar Pac...
9      5.0  Darling Divas Acoustic Steel String Guitar Pac...
10     5.0  Darling Divas Acoustic Steel String Guitar Pac...
11     3.0  Darling Divas Acoustic Steel String Guitar Pac...
12     5.0  Martin D28 Style - Guitar PLANS to Build - Ful...
13     5.0  Martin D28 Style - Guitar PLANS to Build - Ful...
14     3.0  Martin D28 Style - Guitar PLANS to Build - Ful...
15     3

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [4]:
titleDescFixed = []
regex = re.compile('[^a-zA-Z]')
link = '/Users/renswny/Code/R/MIS584/Final Project/stopwords/english'
with open(link) as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

for i in range(len(guitarOnly)):
    temp = regex.sub(' ', guitarOnly.iloc[i,1])
    temp = temp.lower()
    temp = ' '.join([word for word in temp.split() if word not in stopwords])
    titleDescFixed.append(temp)
    
guitarOnly['titleDescFixed'] = titleDescFixed
del guitarOnly['titleDesc']
print(guitarOnly)

    rating                                     titleDescFixed
0      5.0  fender blue ashbory fretless elecric bass guit...
1      4.0  fender blue ashbory fretless elecric bass guit...
2      2.0  fender blue ashbory fretless elecric bass guit...
3      4.0  seavey electronics junior acoustic guitar size...
4      5.0  seavey electronics junior acoustic guitar size...
5      5.0  seavey electronics junior acoustic guitar size...
6      4.0  seavey electronics junior acoustic guitar size...
7      4.0  darling divas acoustic steel string guitar pac...
8      5.0  darling divas acoustic steel string guitar pac...
9      5.0  darling divas acoustic steel string guitar pac...
10     5.0  darling divas acoustic steel string guitar pac...
11     3.0  darling divas acoustic steel string guitar pac...
12     5.0  martin style guitar plans build full scale aco...
13     5.0  martin style guitar plans build full scale aco...
14     3.0  martin style guitar plans build full scale aco...
15     3

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
data_corpus = list(guitarOnly.iloc[:,1])

X = vectorizer.fit_transform(data_corpus)
bagOfWords = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
print(bagOfWords)

    achieved  acoustic  action  active  addition  adjust  adjustable  \
0          0         0       0       1         0       0           0   
1          0         0       0       1         0       0           0   
2          0         0       0       1         0       0           0   
3          0         2       0       0         0       0           1   
4          0         2       0       0         0       0           1   
5          0         2       0       0         0       0           1   
6          0         2       0       0         0       0           1   
7          0         3       0       0         0       0           0   
8          0         3       0       0         0       0           0   
9          0         3       0       0         0       0           0   
10         0         3       0       0         0       0           0   
11         0         3       0       0         0       0           0   
12         0         2       0       0         0       0        

In [35]:
lm = sm.OLS(guitarOnly['rating'], bagOfWords[['acoustic', 'bass', 'vintage', 'rosewood']]).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     27.10
Date:                Sun, 15 Apr 2018   Prob (F-statistic):           2.86e-09
Time:                        19:53:58   Log-Likelihood:                -66.676
No. Observations:                  32   AIC:                             141.4
Df Residuals:                      28   BIC:                             147.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
acoustic       1.7012      0.226      7.512      0.0

In [29]:
posResults = pd.DataFrame({'coefficient': lm.params[(lm.pvalues < 0.05) & (lm.params > 0)]})
negResults = pd.DataFrame({'coefficient': lm.params[(lm.pvalues < 0.05) & (lm.params < 0)]})
print(posResults.sort_values(by = ['coefficient'], ascending = False).to_string())
print(negResults.sort_values(by = ['coefficient']).to_string())

          coefficient
vintage      3.204030
acoustic     1.701166
Empty DataFrame
Columns: [coefficient]
Index: []
